In [1]:
#import modules
import json
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction import text as sk_fe_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score, recall_score
from TwitterAPI import TwitterAPI

In [2]:
#setting up variables for my 'application'
consumerkey = 'f2rs6fF5egLDrd0MMCEC52Iz3'
consumersecret = 'AZ1YB11sp8SOTnrUAWPyTqk0u1YighzijhkKToEM0Cf5BWwgCi'

In [3]:
#setting up the API key
api = TwitterAPI('f2rs6fF5egLDrd0MMCEC52Iz3', 'AZ1YB11sp8SOTnrUAWPyTqk0u1YighzijhkKToEM0Cf5BWwgCi', auth_type = 'oAuth2')

In [4]:
def score(true,pred):
    return (precision_score(true,pred),
            recall_score(true,pred),
            f1_score(true,pred))

In [5]:
def print_score(s):
    print ("""
Precision: {:0.3}
Recall:    {:0.3}
F-SCore:   {:0.3}
""".format(*s))

In [6]:
def searchTwitter(query,feed="search/tweets",api=api,n=2500):
  r = []
  qs = 0
  if len(r)==0:
    r.extend([t for t in api.request("search/tweets",{'q':query,'count':n})])
    qs +=1
  while len(r) < n:
    print("Querrying twitter for {}. {}/{} gathered.".format(query,len(r),n))
    last = r[-1]['id']
    r.extend([t for t in api.request("search/tweets",{'q':query,'count':n,
                                                        'max_id':last})])
    qs += 1
    if qs > 180:
      time.sleep(840)
      qs = 0
  return r[:n]

In [7]:
#Get JSON from Twitter
cats = searchTwitter('#cats')
dogs = searchTwitter('#dogs')

Querrying twitter for #cats. 87/2500 gathered.
Querrying twitter for #cats. 164/2500 gathered.
Querrying twitter for #cats. 242/2500 gathered.
Querrying twitter for #cats. 327/2500 gathered.
Querrying twitter for #cats. 425/2500 gathered.
Querrying twitter for #cats. 517/2500 gathered.
Querrying twitter for #cats. 584/2500 gathered.
Querrying twitter for #cats. 628/2500 gathered.
Querrying twitter for #cats. 693/2500 gathered.
Querrying twitter for #cats. 733/2500 gathered.
Querrying twitter for #cats. 809/2500 gathered.
Querrying twitter for #cats. 888/2500 gathered.
Querrying twitter for #cats. 981/2500 gathered.
Querrying twitter for #cats. 1057/2500 gathered.
Querrying twitter for #cats. 1147/2500 gathered.
Querrying twitter for #cats. 1222/2500 gathered.
Querrying twitter for #cats. 1300/2500 gathered.
Querrying twitter for #cats. 1368/2500 gathered.
Querrying twitter for #cats. 1441/2500 gathered.
Querrying twitter for #cats. 1531/2500 gathered.
Querrying twitter for #cats. 1611/

In [8]:
#convert the json returned by Twitter into a dataframe
cats_d = pd.read_json(json.dumps(cats))
dogs_d = pd.read_json(json.dumps(dogs))

In [9]:
#Get text only and replace  hashtags with blanks
cats_text = [x.replace("#cats", "") for x in cats_d['text']]
dogs_text = [x.replace("#dogs", "") for x in dogs_d['text']]

In [10]:
#Find the number of times #dogs appear in the cats data
blob = [x.find("#dogs") for x in cats_text]
type(blob)
df1 = pd.DataFrame(blob)
df1.stack().value_counts()

-1      2315
 31       29
 80       19
 14       14
 71       14
 106      12
 70       12
 85        6
 51        5
 20        5
 117       5
 15        3
 128       3
 69        3
 118       3
 61        3
 53        2
 23        2
 25        2
 124       2
 39        2
 100       2
 67        2
 58        2
 105       2
 84        1
 76        1
 86        1
 66        1
 90        1
 56        1
 42        1
 96        1
 40        1
 98        1
 30        1
 129       1
 17        1
 108       1
 126       1
 109       1
 103       1
 99        1
 97        1
 89        1
 87        1
 81        1
 79        1
 73        1
 63        1
 59        1
 57        1
 37        1
 121       1
 130       1
 0         1
dtype: int64

In [11]:
#Create features and return sparse metrics 
vectorizer = sk_fe_text.CountVectorizer(cats_text+dogs_text)
vectorizer.fit(cats_text+dogs_text)
cats_tdm = vectorizer.transform(cats_text).toarray()
dogs_tdm = vectorizer.transform(dogs_text).toarray()

In [13]:
#Create visible matrices, combine and add the number of times #dogs appear in cats_text to the length of dog_text and subtract
#the same number from the length of cats_text
zeros = np.zeros((len(cats_text) - 185, 1))
ones = np.ones((len(dogs_text) + 185, 1))
catsdogs = np.concatenate((cats_tdm,dogs_tdm),axis=0)
y = np.ravel(np.concatenate((zeros,ones),axis=0))

In [14]:
#Create train/test split for modeling
trainX,testX,trainY,testY = train_test_split(catsdogs,y,test_size=.20)

In [15]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(trainX,trainY)

GaussianNB(priors=None)

In [16]:
#Naive Bayes cont'd
print("\n\nNaive Bayes Performance")
s = score(testY,nb.predict(testX))
print_score(s)



Naive Bayes Performance

Precision: 0.881
Recall:    0.775
F-SCore:   0.824



In [17]:
#SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(trainX,trainY)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
#SVM Cont'd
print("\n\nSVM performance")
s = score(testY,svm.predict(testX))
print_score(s)



SVM performance

Precision: 0.582
Recall:    0.988
F-SCore:   0.732



In [20]:
#Neural Network
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier()
nn.fit(trainX,trainY)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [21]:
#Neural Network cont'd
print("\n\nNeural Network Performance")
s = score(testY,nn.predict(testX))
print_score(s)



Neural Network Performance

Precision: 0.885
Recall:    0.878
F-SCore:   0.881

